In [1]:
import pandas as pd


In [2]:

# Function to format numbers correctly: 3 significant figures, comma for thousands
def format_number(n):
    if n >= 1e9:
        return f"{n / 1e9:.3g} billion"
    elif n >= 1e6:
        return f"{n / 1e6:.3g} million"
    elif n >= 1e3:
        return f"{int(round(n, -len(str(int(n))) + 3)):,}"  # Rounds to 3 sig figs & formats with commas
    else:
        return str(int(n))

def generate_values_dicts(file_path, sheet_name):
    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Get unique regions
    regions = df["Region"].unique()
    values_dicts = {}

    for region in regions:
        # Get the "Total" row for general values
        region_total = df[(df["Region"] == region) & (df["VF or Non-VF"] == "Total")].iloc[0]

        # Extract VF and Non-VF beneficiary counts correctly
        vf_row = df[(df["Region"] == region) & (df["VF or Non-VF"] == "VF")]
        nonvf_row = df[(df["Region"] == region) & (df["VF or Non-VF"] == "Non-VF")]

        vf_benis = vf_row["Direct Beneficiaries"].values[0] if not vf_row.empty else 0
        nonvf_benis = nonvf_row["Direct Beneficiaries"].values[0] if not nonvf_row.empty else 0

        # Extract Clean Cooking Beneficiaries correctly
        clean_cooking_row = df[(df["Region"] == region) & (df["VF or Non-VF"] == "Total") & (df["Subcategory"] == "Clean Cooking")]
        clean_cooking_benis = clean_cooking_row["Direct Beneficiaries"].values[0] if not clean_cooking_row.empty else 0

        # Use "Energy Access" directly for clean electricity
        clean_electricity_benis = max(region_total["Energy Access"] - clean_cooking_benis, 0)  # Ensure it's not negative

        # Extract the top 3 beneficiary categories (ONLY from "Beneficiary Tier" and excluding specific subcategories)
        category_data = df[
            (df["Region"] == region) & 
            (df["VF or Non-VF"] == "Total") & 
            (df["Category"] == "Beneficiary Category") & 
            (~df["Subcategory"].isin(["Clean Cooking", "Energy (MW added)", "Electricity Access", "Other Energy Services"]))
        ]
        category_totals = category_data.set_index("Subcategory")["Direct Beneficiaries"].to_dict()

        # Sort by highest values and pick the top 3
        sorted_categories = sorted(category_totals.items(), key=lambda x: x[1], reverse=True)
        top_categories = dict(sorted_categories[:3])

        # Extract "indirect-projects" from row where "Category" is "Beneficiary Tier" and "Subcategory" is "Policy and Regulator Frameworks"
        indirect_row = df[
            (df["Region"] == region) & 
            (df["Category"] == "Beneficiary Tier") & 
            (df["Subcategory"] == "Policy and Regulator Frameworks")  # Uses the misspelled version
        ]
        indirect_projects = indirect_row["Project Count"].values[0] if not indirect_row.empty else 0

        if region == "All":
            region_title = "globally"  # Leave blank for "All"
        elif region in ["LDC", "LLDC"]:
            region_title = "in "+ region + "s"  # Make plural: "LDCs", "LLDCs"
        else:
             region_title = "in "+ region # Default case: "in SIDS", "in Africa", etc.
        
        # Construct the dictionary for the region
        values_dicts[region] = {
            "region-title": region_title,
            "number-of-projects": format_number(region_total["Project Count"]),
            "number-of-countries": format_number(region_total["Country Count"]),
            "budget-text": format_number(region_total["Budget Sum (M USD)"]),  # No more "million"
            "direct-benis": format_number(region_total["Direct Beneficiaries"]),
            "vf-benis": format_number(vf_benis),
            "nonvf-benis": format_number(nonvf_benis),
            "clean-electricity-benis": format_number(clean_electricity_benis),
            "clean-cooking-benis": format_number(clean_cooking_benis),
            "productive-benis": format_number(region_total["Productive Use"]),
            "top-categories": {k: format_number(v) for k, v in top_categories.items()},  # Sorted dict
            "top-categories-text": ", ".join(top_categories.keys()),
            "indirect-projects": format_number(indirect_projects),
        }

    return values_dicts

def generate_summary(region_data):
    """Generate the summary text using the extracted region data."""
    summary = (
        f"The UNDP portfolio {region_data['region-title']} during the 2022 to 2025 Strategic Plan includes "
        f"{region_data['number-of-projects']} active energy-related projects across {region_data['number-of-countries']} countries. "
        f"The total budget for these projects is {region_data['budget-text']} USD, targeting {region_data['direct-benis']} direct beneficiaries. "
        f"This includes {region_data['vf-benis']} from VF projects and {region_data['nonvf-benis']} from non-VF projects.\n\n"
        f"Of those directly benefiting, {region_data['clean-electricity-benis']} are gaining access to clean electricity, "
        f"and {region_data['clean-cooking-benis']} are gaining access to clean cooking. Additionally, "
        f"{region_data['productive-benis']} people benefit from productive uses of energy, including "
    )

    # Add top categories
    top_category_items = [f"{v} in {k}" for k, v in region_data["top-categories"].items()]

    if len(top_category_items) > 1:
        top_categories_text = ", ".join(top_category_items[:-1]) + ", and " + top_category_items[-1]
    else:
        top_categories_text = top_category_items[0] if top_category_items else ""

    if top_categories_text:
        summary += top_categories_text + ". "


    # Add indirect projects
    summary += (
        f"{region_data['indirect-projects']} projects also include outputs related to supporting policy or regulatory frameworks, "
        f"capacity building, or other system benefits that provide indirect benefits."
    )

    return summary

# Set file path and sheet name
file_path = "../02_Output/00_Final Results/Moonshot Tracker Results - Auto.xlsx"
sheet_name = "Summary"

# Generate values dictionary
values_dicts = generate_values_dicts(file_path, sheet_name)

# Generate and print summaries for all regions
for region, region_data in values_dicts.items():
    print(generate_summary(region_data))
    print("\n" + "-" * 100 + "\n")


The UNDP portfolio globally during the 2022 to 2025 Strategic Plan includes 398 active energy-related projects across 132 countries. The total budget for these projects is 4.23 billion USD, targeting 82.5 million direct beneficiaries. This includes 16.8 million from VF projects and 65 million from non-VF projects.

Of those directly benefiting, 52.4 million are gaining access to clean electricity, and 2.73 million are gaining access to clean cooking. Additionally, 27.3 million people benefit from productive uses of energy, including 11.4 million in Health Services, 5.36 million in Water Services, and 4.39 million in Transportation and E-mobility Services. 108 projects also include outputs related to supporting policy or regulatory frameworks, capacity building, or other system benefits that provide indirect benefits.

----------------------------------------------------------------------------------------------------

The UNDP portfolio in RBLAC during the 2022 to 2025 Strategic Plan i